In [1]:
from selenium import webdriver
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#link the url to the web driver
driver = webdriver.Chrome() #Selenium usage through WebDriver
url = "https://volcano.si.edu/search_eruption.cfm"
driver.get(url) #WebDriver gets url se we can click on elements in the URL

In [3]:
searchclick = driver.find_element(By.XPATH, "//*[@id='submit_account']").click()
#Clicks the Search button in the url mentioned

In [4]:
def get_num_of_pages():
    src= driver.page_source #to get the HTML code of the current page
    parser = BeautifulSoup(src, "lxml") #we convert from xml to lxml
    page_bar = parser.find("div", attrs = {"class":"paginate"})
    num_of_li = page_bar.findAll('li')#to get the num of pages
    num_of_pages = len(num_of_li) - 2 #we subtract 2 because it includes the next and previous button in the num_of_li.
    return num_of_li

In [5]:
def get_volc_details(index):
    Real_Summit = ''
    click_on_name = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, f"//*[@id='content']/div/div[4]/div/table/tbody/tr[{index}]/td[1]/a"))).click()
    #WebDriverWait.until means the driver is loading up until 30 seconds until its clickable 
    time.sleep(3)
    driver.execute_script('window.stop();')
    Country = driver.find_element(By.XPATH, "//*[@id='ProfileHolocene']/div[4]/ul[1]/li[1]").text #taking all information needed by XPATH used by selenium
    Volcanic_Region = driver.find_element(By.XPATH, "//*[@id='ProfileHolocene']/div[4]/ul[1]/li[2]").text
    Volcano_type = driver.find_element(By.XPATH, "//*[@id='ProfileHolocene']/div[4]/ul[1]/li[3]").text
    Latitude = driver.find_element(By.XPATH, "//*[@id='ProfileHolocene']/div[5]/ul[1]/li[1]").text
    Longitude = driver.find_element(By.XPATH, "//*[@id='ProfileHolocene']/div[5]/ul[1]/li[2]").text
    Summit = driver.find_element(By.XPATH, "//*[@id='ProfileHolocene']/div[5]/ul[1]/li[4]").text
    for k in range(Summit.find('m')): #tutrning the summit into int instead of string
        Real_Summit += Summit[k]
    return Country,Volcanic_Region,Volcano_type,Latitude,Longitude,int(Real_Summit)

In [6]:
def page_table_to_df(): # a function that turns the i page table to a dataframe
    headerList = []
    src = driver.page_source 
    parser = BeautifulSoup(src, "lxml") 
    table = parser.find("div", attrs = {"class":"TableSearchResults"}) 
    headers = table.findAll('th') #to find headers of table
    for h in headers: 
        headerList.append(h.text.strip())
    rows = table.findAll('tr')#all rows
    cols = [[td.getText().strip() for td in rows[i].findAll('td')]for i in range(len(rows))]
    volcano = pd.DataFrame(cols, columns = headerList) #turning into dataframe
    volcano.drop(columns=[''], inplace = True)#there was a NaN column left behind the code so we are dropping it
    volcano.drop([0],inplace = True)
    return volcano

In [7]:
def get_num_of_rows(page_num):
    page = driver.find_element(By.XPATH,f"//*[@id='content']/div/div[3]/ul/li[{page_num}]/a").click()
    src = driver.page_source 
    parser = BeautifulSoup(src, "lxml") 
    table = parser.find("div", attrs = {"class":"TableSearchResults"})
    rows = table.findAll('tr')
    return rows

In [8]:
def unkown_source_nan_values():#unkown sources will get NULL values
    Country=None
    Volcanic_Region=None
    Volcano_type=None
    Latitude=None
    Longitude=None
    Summit=None
    return Country,Volcanic_Region,Volcano_type,Latitude,Longitude,Summit

In [10]:
Mountain = []#first iteration, we simply setting the first values
Mountain_dataframe_columns = "Country","Volcanic_Region","Volcano_type","Latitude","Longitude","Summit"
page_table_df=page_table_to_df()
page_num=2 # 2 means page 1, 3 means page 2 ....
num_of_pages=get_num_of_pages()
row_index = 1

The Amount Of Pages:  15


In [ ]:
while(page_num!= len(num_of_pages)):
    rows = get_num_of_rows(page_num)
    for row in rows:
        Name = driver.find_element(By.XPATH, f"//*[@id='content']/div/div[4]/div/table/tbody/tr[{row_index}]/td[1]/a").text
        if(Name != "Unknown Source"):#we input the i row volcano name to identify unkown sources
            print(Name)
            Mountain.append((get_volc_details(row_index))) #inserting the volcano details to a array 
            driver.execute_script("window.history.go(-1)")#go a page back
            row_index=row_index+1
            if(row_index == len(rows)):#to check if its the last row
                if(page_num!= 2):#so we dont concat the first page dataframe to itself
                    page_table_df = page_table_to_df()
                row_index = 1
                page_num+=1
                Mountain_df = pd.DataFrame(Mountain , columns = Mountain_dataframe_columns) #converting the i page data we got to a data frame 
                Mountain_df.index = Mountain_df.index+1
                Mountain =[]
                result = pd.concat([page_table_df,Mountain_df],axis = 1,ignore_index = False)#concating both dataframes
                result.to_csv("main_df_csv.csv",mode = 'a', index = False,header = False)
                result = pd.DataFrame()      
        else: #for unkown sources.
            Mountain.append(unkown_source_nan_values())
            row_index=row_index+1
            if(row_index == len(rows)):
                if(page_num!= 2):
                    page_table_df = page_table_to_df()
                page_num+=1
                row_index=1
                rows = get_num_of_rows(page_num)
                Mountain_df = pd.DataFrame(Mountain , columns = Mountain_dataframe_columns)
                Mountain_df.index = Mountain_df.index+1
                Mountain =[]
                result = pd.concat([page_table_df,Mountain_df],axis = 1,ignore_index = False)
                result.to_csv("main_df_csv.csv",mode = 'a', index = False,header = False)
                result = pd.DataFrame()

In [14]:
main_df = pd.read_csv("main_df_csv.csv") #SAVING THE DATAFRAME
main_df.index = main_df.index + 1
main_df

,Volcano Name,Eruption Type,Start Date,Max. VEI,Country,Volcanic_Region,Volcano_type,Latitude,Longitude,Summit
1,Ambae,Confirmed Eruption,2021 Dec 5,--,Vanuatu,Melanesia and Australia,Shield,15.389°S,167.835°E,1496.0
2,Pinatubo,Confirmed Eruption,2021 Nov 30,--,Philippines,Philippines and SE Asia,Stratovolcano,15.13°N,120.35°E,1486.0
3,Iliwerung,Confirmed Eruption,2021 Nov 28,--,Indonesia,Indonesia,Complex,8.532°S,123.573°E,583.0
4,Taal,Confirmed Eruption,2021 Nov 15,--,Philippines,Philippines and SE Asia,Caldera,14.002°N,120.993°E,311.0
5,Turrialba,Confirmed Eruption,2021 Nov 3,--,Costa Rica,México and Central America,Stratovolcano,10.025°N,83.767°W,3340.0
...,...,...,...,...,...,...,...,...,...,...
11218,"Yojoa, Lago",Confirmed Eruption,1073 BCE,--,Honduras,México and Central America,Volcanic field,14.964°N,87.983°W,1060.0
11219,Quetrupillan,Confirmed Eruption,1345 BCE,3,Chile,South America,Stratovolcano,39.496°S,71.722°W,2360.0
11220,Saunders,Confirmed Eruption,NaN,0,United Kingdom,Antarctica,Stratovolcano,57.8°S,26.483°W,843.0
11221,"Fournaise, Piton de la",Confirmed Eruption,NaN,--,France,Middle East and Indian Ocean,Shield,21.244°S,55.708°E,2632.0
